In [2]:
import unittest
import requests
import pandas as pd
import numpy as np
import os.path
from FIFAApp import *
from FIFADataWebscrapper import *

class Fifatestcase(unittest.TestCase):
    
    # Test request to site is successful
    def test_webrequest(self):
        response = requests.get('https://www.futbin.com/20/players?page=1&sort=Player_Rating&order=desc&version=gold')
        self.assertEqual(response.status_code, 200)
        
    # Test if csv file was written
    def test_webscrapper(self):
        self.assertTrue(os.path.exists('FIFA Player Info.csv'))
    
    # Test the structure of csv file
    def test_fifastructure(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        self.assertEqual(df.dtypes.tolist(), df.dtypes.tolist())
    
    # Test the structure of csv file    
    def test_fifastructure2(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        self.assertEqual(len(df.columns), 21)
        self.assertEqual(len(df), 16359)
    
    # Test position group support function
    def test_positiongroup(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        df['Position'].apply(position)
        self.assertListEqual(df['Position Group'].unique().tolist(), ['Attacker', 'Midfielder', 'Goal Keeper', 'Defender'])
    
    # Test continent code support function
    def test_continentcode(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        df['Country'].head(100).apply(continent)
        self.assertEqual(df['Continent'].isna().sum(), 0)
    
    # Test for duplicates
    def test_duplicates(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        dup = df.duplicated(subset = ['Name', 'Overall Rating', 'Position'])
        self.assertEqual(dup.value_counts().tolist()[0], 16359)
    
    # Test select function for scatter plot
    def test_select_scatter(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        overall_rating_val = [df['Overall Rating'].min(), df['Overall Rating'].max()]
        skill_level = [df['Skill'].min(), df['Skill'].max()]
        self.assertEqual(len(select_fifa()), 16359)
    
    # Test select histogram function
    def test_select_histogram(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected, skill = select_hist()
        self.assertEqual(skill, "Passing")
        
    # Test select best starting eleven
    def test_select_best(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected = select_fifa2()
        self.assertEqual(len(selected), 11)
    
    # Test select best starting eleven contains Lionel Messi
    def test_select_best2(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected = select_fifa2()
        messi = selected[selected.Name.str.contains('Lionel Messi')==True]
        self.assertEqual(messi.Name[0], 'Lionel Messi')
    
    # Test select number of defenders in best starting eleven 
    def test_select_best3(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected = select_fifa2()
        num_def = selected[selected['Position Group'] == 'Defender']
        self.assertEqual(len(num_def), 4)
    
    # Test if only one goalie is picked
    def test_bestplayers_singleGoalie(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '442', 'Overall Rating')
        goalie_num = len(best[best['Position Group'] == 'Goal Keeper'])
        self.assertEqual(goalie_num, 1)
    
    # Test if number of defenders are equal to formation selected
    def test_bestplayers_defenders(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '532', 'Defending')
        def_num = len(best[best['Position Group'] == 'Defender'])
        self.assertEqual(def_num, 5)
        
    # Test if number of midfielders are equal to formation selected
    def test_bestplayers_midfielder(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '442', 'Overall Rating')
        mf_num = len(best[best['Position Group'] == 'Midfielder'])
        self.assertEqual(mf_num, 4)
    
    # Test if number of attackers are equal to formation selected
    def test_bestplayers_attacker(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '442', 'Shooting')
        atk_num = len(best[best['Position Group'] == 'Attacker'])
        self.assertNotEqual(atk_num, 3)
    
    # Test if any formation yeilds 433 formation
    def test_bestformation(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestformation(df, 'Any', 'Overall Rating')
        atk_num = len(best[best['Position Group'] == 'Attacker'])
        mid_num = len(best[best['Position Group'] == 'Midfielder'])
        def_num = len(best[best['Position Group'] == 'Defender'])
        self.assertEqual(atk_num, 3)
        self.assertEqual(mid_num, 3)
        self.assertEqual(def_num, 4)
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
        

.................
----------------------------------------------------------------------
Ran 17 tests in 4.132s

OK
